In [6]:
# IMPORT BLOCK:____________________________________________

import numpy as np
from numpy import random as npr
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import animation as animation
from matplotlib import colors as matcol
import copy
import itertools
# import itertools as it

npr.seed(69420)

In [7]:
# INITIAL DATA:_____________________________________________
cooperate = np.array( [ 1, 0 ] )
defect = np.array( [ 0, 1 ] )

# default Payoff matrix
default_R = 2
default_S = 0
default_T = 3
default_P = 1
Payoff = np.array( [ [ default_R , default_S ], [ default_T, default_P ] ] )


In [8]:
# # OLD STRATEGIES:________________________________________________

# # defining all strategies
# # # template function
# # parameter_list = [ k, ]
# # def strategy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
# #     # move either cooperate or defect
# #     move
# #     return move 

# # nice guy function
# def nice_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     move = cooperate
#     return move 

# # bad guy function
# def bad_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     move = defect
#     return move 

# # mainly nice guy function
# def mainly_nice( parameter_list = [ 0.4 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     k = parameter_list[ 0 ]
#     if k > 0.5 : 
#         print( "Wrong function called: this is mainly_nice" )
#         return - 1
#     u = npr.random( )
#     if ( u < k ) : 
#         move = defect
#     else : 
#         move = cooperate
#     return move

# # mainly bad guy function
# def mainly_bad( parameter_list = [ 0.6 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     k = parameter_list[ 0 ]
#     if k < 0.5 : 
#         print( "Wrong function called: this is mainly_bad" )
#         return - 1
#     u = npr.random( )
#     if ( u < k ) : 
#         move = defect
#     else : 
#         move = cooperate
#     return move

# # tit_for_tat function
# def tit_for_tat( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     if ( round_number == 0 ) : 
#         move = cooperate
#     else : 
#         move = match_history[ round_number - 1 ][ 1 - player_index ]
#     return move

# # resentful guy function
# def resentful_guy( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     move = cooperate
#     for res in match_history :
#         if np.array_equal( res[ 1 - player_index  ], defect ) :
#             move = defect
#             break
#     return move

# # trusting guy (increasing odds of cooperating according to match history)
# def trusting_guy( parameter_list = [ 0.5 ], round_number = 0, match_history = [[]], player_index = 0 ) :
#     # starting trust
#     k = parameter_list[ 0 ]
#     # environmental trust
#     if ( round_number > 0 ) :
#         coop = 0. 
#         for res in match_history : coop += res[ 1 - player_index  ][ 0 ]
#         k = coop / len( match_history )
#     u = npr.random( )
#     if ( u > k ) : 
#         move = defect
#     else : 
#         move = cooperate
#     return move

# # balancing guy (cooperates only for an even cooperation history)
# def balancing_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :
#     move = defect
#     if ( round_number > 0 ) :
#         # total cooperation seen
#         coop = 0
#         for res in match_history : coop += ( res[ 0 ][ 1 ] + res[ 1 ][ 1 ])
#         if ( ( coop % 2 ) == 0 ) : 
#             move = cooperate
#     return move

# # dictionary of all strategies
# strategies = {
#     'NiceGuy' : nice_guy,
#     'BadGuy' : bad_guy,
#     'MainlyNice' : mainly_nice,
#     'MainlyBad' : mainly_bad,
#     'TitForTat' : tit_for_tat,
#     'ResentfulGuy' : resentful_guy,
#     'TrustingGuy' : trusting_guy,
#     'Thanos' : balancing_guy,
# }

In [9]:
# NEW STRATEGIES:________________________________________________

# defining all strategies
# # template function
# parameter_list = [ k, ]
# def strategy( round_number = 0, match_history = [[]], player_index = 0 ) : 
#     # move either cooperate or defect
#     move
#     return move 

# nice guy function
def nice_guy( round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    return move 

# bad guy function
def bad_guy( round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = defect
    return move 

# percentage of defect for mainly nice ( and 1-k percentage of cooperate for mainly bad) Global Variable
k = 0.25

# mainly nice guy: randomly defect k% of the times and cooperate 100-k %, k<50
def mainly_nice( round_number = 0, match_history = [[]], player_index = 0 ) : 
    u = npr.random( )
    if ( u < k ) : move = defect
    else :         move = cooperate
    return         move

# mainly bad guy: randomly defect k% of the times and cooperate 100-k %, k>50
def mainly_bad( round_number = 0, match_history = [[]], player_index = 0 ) : 
    u = npr.random( )
    if ( u > k ) : move = defect
    else :         move = cooperate
    return         move

# random guy: randomly defect or cooperate
def random_guy( round_number = 0, match_history = [[]], player_index = 0 ) : 
    u = npr.random( )
    if ( u < 0.5 ) : move = defect
    else :           move = cooperate
    return           move

# tit_for_tat function:  start by cooperating, then repeat what the opponent has done in the previous move
def tit_for_tat( round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : move = cooperate
    else :                     move = match_history[ round_number - 1 ][ 1 - player_index ]
    return                     move

# resentful guy function: cooperate until the opponent defects once, then always defect
def resentful_guy( round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    if ( round_number > 0 ) :
        for res in match_history : 
            # if any result matching a defect is found, the move is set to defect
            if np.array_equal( res[ 1 - player_index  ], defect ) : move = defect
            # if a defect is found, he would break    
            break
    return move

# trusting guy (increasing odds of cooperating according to match history):
# initially is like random guy (coop/defect at k=50%), then updates k
def trusting_guy( round_number = 0, match_history = [[]], player_index = 0 ) :
    # starting trust
    k_t = 0.5
    # environmental trust
    if ( round_number > 0 ) :
        # number of cooperative behaviour met
        coop = 0. 
        # increase the value of cooperation by one each cooperation (element 0) seen from the other player (index 1-player_index)
        for res in match_history : coop += res[ 1 - player_index  ][ 0 ]
        # normalizing result over the match history
        k_t = coop / len( match_history )
    u = npr.random( )
    if ( u > k_t ) : move = defect
    else :           move = cooperate
    return           move

# balancing guy (cooperates only for an even cooperation history)
def balancing_guy( round_number = 0, match_history = [[]], player_index = 0 ) :
    move = defect
    if ( round_number > 0 ) :
        # total cooperation seen
        coop = 0
        # increase the value of cooperation by one each cooperation (element 0) seen from each side during the match
        for res in match_history : coop += ( res[ 0 ][ 1 ] + res[ 1 ][ 1 ])
        if ( ( coop % 2 ) == 0 ) : move = cooperate
    return                         move

# Mid resentful: cooperates on the first move, and defects if the opponent has defected on any of the previous 3 moves, else cooperates
def mid_resentful( round_number = 0, match_history = [[]], player_index = 0):
    move = cooperate
    # selecting the oldest round to check, 3 before the current one if possible, the starting round otherwise
    start = max(0, round_number - 3)
    for round_i in range( start, round_number ):
        # checking if one of the previous move of the opponent (1-player_index) mach a defect
        if np.array_equal( match_history[round_i][1 - player_index], defect ):   
            move = defect
            # if a result is found, the cycle can break
            break
    return move

# reverse tit_for_tat function: start by defecting, then does the opposite of what the opponent has done in the previous move
def reverse_tit_for_tat( round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : move = defect
                               # in this way i obtain the opposite move: [1,1] - [1,0] = [0,1] and [1,1] - [0,1] = [1,0]
    else :                     move = [1,1] - match_history[ round_number - 1 ][ 1 - player_index ]  
    return                     move

# learning to scam guy: start by defecting, then cooperating two times; for the following turns chose the optimal condition:
# if the opponent never defected, he will defect to scam the opponent
# if the opponent cooperated while he cooperated more than he was scammed, he will cooperate, because he sees some kind of intelligence
# otherwise he'll defect to avoid beeing scammed
def scamming_guy( round_number = 0, match_history = [[]], player_index = 0 ) :
    # default move is defect
    move =  defect
    # selecting first tree moves
    if ( round_number == 0 )   : 
        # print( f"round {round_number}: I defect" )
        move = defect
    elif ( round_number == 1 ) : 
        # print( f"round {round_number}: I cooperate" )
        move = cooperate
    elif ( round_number == 2 ) : 
        # print( f"round {round_number}: I cooperate" )
        move = cooperate
    else :
        # counting number of matching cooperation isses and defected from 0
        matching_coop = 0
        scammed = 0
        # deciding how mutch history to check
        first_r = npr.randint( 3, ( round_number + 1 ), dtype = int )
        # print( first_r )
        # searching over match history
        for i in range( round_number ) :
            # checking if the result cooperate, cooperate occurs
            if( np.array_equal( match_history[i][0], cooperate ) and np.array_equal( match_history[i][1], cooperate ) ) :
                matching_coop += 1
            # checking if the result cooperate, defect occurs in favour of the opposite side
            elif( np.array_equal( match_history[i][1 - player_index], defect ) and np.array_equal( match_history[i][player_index], cooperate ) ) :
                scammed += 1
        # checking if the opponent ever defected
        if( np.all( np.array_equiv( np.array( [ match_history[i][ 1 - player_index ] for i in range( ( round_number - first_r ), round_number ) ] ) , cooperate ) ) ) :
            # print( f"The rival is too good, I defect" )
            move = defect
        elif( matching_coop >= scammed ) : 
            # print( f"The rival is smart, I cooperate" )
            move = cooperate
    return move

# dictionary of all strategies
strategies = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'MainlyBad' : mainly_bad,
    'TitForTat' : tit_for_tat,
    'RandomGuy' : random_guy,
    'ResentfulGuy' : resentful_guy,
    'TrustingGuy' : trusting_guy,
    'Thanos' : balancing_guy,
    'MidResentful': mid_resentful,
    'ReverseTft': reverse_tit_for_tat,
    'ScammingGuy' : scamming_guy,
}

In [10]:
# # OLD MATCH DEFINING FUNCTIONS:________________________________________________

# # function to play one round 1 vs 1 for two different* strategies players
# def round( key_1, key_2, parameter_list_1, parameter_list_2, round_number, match_history, M = Payoff ) :
#     # getting strategies from dictionary
#     S_1 = strategies[ key_1 ]
#     S_2 = strategies[ key_2 ]
#     # computing next move for each player
#     u_1 = S_1( parameter_list_1, round_number, match_history, 0 )
#     u_2 = S_2( parameter_list_2, round_number, match_history, 1 )
#     # computing rewards for each player
#     r_1 = np.dot( u_1, Payoff.dot(u_2))
#     r_2 = np.dot( u_2, Payoff.dot(u_1))
#     # updating match history
#     match_history.append( [ u_1, u_2 ] )
#     # returning res1 - player_indexults
#     return r_1, r_2

# # function to play a match of N rounds 1 vs 1 for two different* strategies player
# def match( key_1, key_2, parameter_list_1, parameter_list_2, N_rounds = 10, M = Payoff ) : 
#     # match history strarting empty
#     history = [ ]
#     # starting total rewards
#     R_1 = 0
#     R_2 = 0
#     # list of partial sums
#     Rewards = []
#     for round_i in range( 0, N_rounds ) :
#         r_1, r_2 = round( key_1, key_2, parameter_list_1, parameter_list_2, round_i, history, M )
#         R_1 += r_1
#         R_2 += r_2
#         Rewards.append( [ R_1, R_2 ] )
#     return Rewards


In [11]:
# NEW MATCH DEFINING FUNCTIONS:________________________________________________

# function to play one round 1 vs 1 for two different* strategies players
def round( key_1, key_2, round_number, match_history, M = Payoff ) :
    # computing minimum and maximum reward for normalization
    s = np.dot( cooperate, M.dot( defect ) )
    t = np.dot( defect, M.dot( cooperate ) )
    # getting strategies from dictionary
    S_1 = strategies[ key_1 ]
    S_2 = strategies[ key_2 ]
    # computing next move for each player
    u_1 = S_1( round_number, match_history, 0 )
    u_2 = S_2( round_number, match_history, 1 )
    # computing rewards for each player
    r_1 = np.dot( u_1, M.dot(u_2))
    r_2 = np.dot( u_2, M.dot(u_1))
    # normalizing results
    r_1 = float( r_1 - s ) / float( t - s )
    r_2 = float( r_2 - s ) / float( t - s )
    # updating match history
    match_history.append( [ u_1, u_2 ] )
    # returning results
    return r_1, r_2

# function to play a match of N rounds 1 vs 1 for two different* strategies player
def match( key_1, key_2, N_rounds = 10, M = Payoff ) : 
    # match history strarting empty
    history = [ ]
    # starting total rewards
    R_1 = 0
    R_2 = 0
    # list of partial sums
    Rewards = []
    for round_i in range( 0, N_rounds ) :
        r_1, r_2 = round( key_1, key_2, round_i, history, M )
        R_1 += r_1
        R_2 += r_2
        # Rewards.append( [ R_1, R_2 ] )
        # normalizing results to round played up this moment
        Rewards.append( [ float(R_1)/( round_i + 1. ), float(R_2)/( round_i + 1. ) ] )
    # returns the history list of partial sums of rounds rewards
    return Rewards
